# AsyncIO
AsyncIO is a concurrent programming design which sometimes we associate with concepts as *concurrency, parallelism, threading, multiprocessing, etc*. Then let's start with some definitions:
- parallelism: perform multiple operations at the same time.
- multiprocessing: is a way to perform parallelism where tasks are spread over CPUs or cores
- concurrency: perform multiple tasks in an overlapped manner. It is a bigger concept thatn parallelism and concurrency does not imply parallelism
- threading: concurrent execution model where multiples threads takes turns executing tasks

Knowing the above concepts, **asyncIO is a library to write concurrent code, but is not threading or multiprocessing**.
It is a single-threaded, single-process design that uses **cooperative multitasking**.
The difference between AsyncIO and threading is that:
- threading: operative system controls the change between threads, then the code doesn't need to do anything to make them switch but it can be difficult because it can change anytime.
- AsyncIO: programmer constrols the change between tasks, then it is expected cooperative multitasking: the tasks must announce when they are ready to be switched out. This implies extra work but we know exactly where the task will be swapped out.

Concurrency and therefore AsyncIO is useful when our program must wait for external sources, for example, http requests, upload/download files, printer, etc. Then, instead of waste time waiting for the external source, we pause the current task and advance with another one, then we go back to the first task to see if the external source is ready or we continue with something else.

Summaryzing, AsyncIO uses one thread and one core, and instead of waiting for external entities, it does something else.

For basic usage we will be using:
- **async** is used to define a native coroutine or an asynchronous generator: async def, async with or async for.
- **await** passes the control back to the event loop while waiting for a result.

In [15]:
import asyncio
import time

def greet(name: str):
    print("[sync]", "Hello")
    time.sleep(1)
    print("[sync]", name)

async def greet_async(name: str):
    print("[async]", "Hello")
    # We execute the first part of the function, then give the control back to the loop to execute something else instead of waiting
    await asyncio.sleep(1)
    print("[sync]", name)

# Sync approach takes 3 seconds
s = time.perf_counter()
greet("Juan")
greet("Sebastian")
greet("Andres")
print(f"Sync executed in: {time.perf_counter() - s}")

# Async approach takes 1 second because instead of waiting in the first greet_async, the loop continue with the next one
s = time.perf_counter()
async def main():
    await asyncio.gather(greet_async("Juan"), greet_async("Sebastian"), greet_async("Andres"))
## In Jupyter notebooks we can await async functions without assigning it to a loop
await main()
print(f"Async executed in: {time.perf_counter() - s}")


[sync] Hello
[sync] Juan
[sync] Hello
[sync] Sebastian
[sync] Hello
[sync] Andres
Sync executed in: 3.003924213000573
[async] Hello
[async] Hello
[async] Hello
[sync] Juan
[sync] Sebastian
[sync] Andres
Async executed in: 1.0031405080007971


We can chain coroutines waiting for a coroutine result and use it in another one.

In [20]:
import random
import asyncio

async def get_temperature():
    print("[get_temperature]", "Obtaining temperature from our amaizing towers!")
    await asyncio.sleep(1)
    return random.randint(10, 40)

async def wear_jacket(temperature):
    print("[wear_jacket]", "Calculating if you should wear a jacket!")
    await asyncio.sleep(1)
    return temperature < 20

async def main():
    print("[main]", "Starting program")
    temperature = await get_temperature()
    should_wear_jacket = await wear_jacket(temperature)
    msg = "Wear a jacket!" if should_wear_jacket is True else "Don't wear a jacket!"
    print("Temperature:", temperature, msg)
    
# Runtime of the main function is equalts to the sum of the called tasks runtime
await main()
    

[main] Starting program
[get_temperature] Obtaining temperature from our amaizing towers!
[wear_jacket] Calculating if you should wear a jacket!
Temperature: 33 Don't wear a jacket!


When we call a coroutine, we obtain a coroutine object, it is not actually execute. To execute an async coroutine we can use:
- **asyncio.run()**
- **asyncio.create_task()** takes a coroutine an optional name and returns a **Task**.

In [25]:
import asyncio
import time
import random

async def greet_async(wait, name: str):
    print("[async]", "Hello")
    await asyncio.sleep(wait)
    print("[async]", name)

async def main():
    s = time.perf_counter()
    await greet_async(1, "Juan")
    await greet_async(2, "Sebastian")
    print("[main]", f"finished in {time.perf_counter() - s}")
    
async def main_create_task():
    s = time.perf_counter()
    task1 = asyncio.create_task(greet_async(1, "Juan"))
    task2 = asyncio.create_task(greet_async(2, "Sebastian"), name="greetSebastian")

    # Wait until both tasks are completed (should take around 2 seconds.)
    await task1
    await task2

    print("[main_create_task]", f"finished in {time.perf_counter() - s}")

await main()
await main_create_task()

[async] Hello
[async] Juan
[async] Hello
[async] Sebastian
[main] finished in 3.0047011130009196
[async] Hello
[async] Hello
[async] Juan
[async] Sebastian
[main_create_task] finished in 2.002152148997993
